In [1]:
import tensorflow as tf
import os
import scipy.io
import numpy as np

2024-04-12 18:52:46.287151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 18:52:46.287217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 18:52:46.452729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 18:52:46.812177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 18:52:48.331157: W tensorflow/compiler/tf2

In [2]:
working_dir = os.getcwd()

file_path = os.path.join(working_dir, 'jaxpi', 'examples', 'allen_cahn','data', 'allen_cahn.mat')

data = scipy.io.loadmat(file_path)

In [3]:
# get data for model
x = tf.convert_to_tensor(data['x'].flatten()[:,None])
t = tf.convert_to_tensor(data['t'].flatten()[:,None])

usol = (np.real(data['usol']))
usol = tf.reshape(usol, shape=(1,201,512))

2024-04-12 18:53:05.881218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 18:53:06.393633: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 18:53:06.393686: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 18:53:06.397768: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-12 18:53:06.397828: I external/local_xla/xla/stream_executor

In [7]:
print(len(x))

512


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(1, name='output'),
])

In [ ]:
def train()

In [61]:
with tf.GradientTape(persistent=True) as tapex, tf.GradientTape(persistent=True) as tapet:
    tapex.watch(x)
    tapet.watch(t)

    inputs = tf.concat([t,x],axis=0)
    inputs = tf.transpose(inputs)

    model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])
    model.fit(x=inputs,y=usol, epochs=50)

    # calculate gradients
    y = model(inputs)

    print(y)

    u_t = tapet.gradient(y, t, unconnected_gradients='zero')
    u_x = tapex.gradient(y, x, unconnected_gradients='zero')

    print(u_t)
    print(u_x)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.1095 - loss: 3.2953e-06
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.1393 - loss: 1.1961e-05
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.0896 - loss: 1.1375e-05
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.0896 - loss: 4.3494e-06
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - accuracy: 0.1095 - loss: 5.4971e-06
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - accuracy: 0.1194 - loss: 4.9095e-06
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.1045 - loss: 1.3865e-06
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.0945 - loss: 6.2478e-06
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.1095 - loss: 1.2785e-06
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step - accuracy: 0.1393 - loss: 2.0619e-06
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.1592 - loss: 3.6515e-06
Epoch 12/100
1/1 ━━━━━

In [99]:
model.evaluate(x=inputs, y=usol)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.6816 - loss: 1.2751e-14


[1.2750531797652963e-14, 0.6815920472145081]

In [250]:
# calculate gradient
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    forward = model(x)
    u_x = tape.gradient(forward, x, unconnected_gradients='zero')
    u_xx = tape.gradient(u_x, x, unconnected_gradients='zero')

#print(forward)
print(u_x)
print(u_xx)

tf.Tensor(
[[ 2.43167830e+00]
 [ 2.43249726e+00]
 [ 2.43330884e+00]
 [ 2.43411350e+00]
 [ 2.43491054e+00]
 [ 2.43570018e+00]
 [ 2.43648291e+00]
 [ 2.43725777e+00]
 [ 2.43802547e+00]
 [ 2.43878603e+00]
 [ 2.43953919e+00]
 [ 2.44028473e+00]
 [ 2.44102311e+00]
 [ 2.44175434e+00]
 [ 2.44247746e+00]
 [ 2.44319344e+00]
 [ 2.44390202e+00]
 [ 2.44460297e+00]
 [ 2.44529676e+00]
 [ 2.44598293e+00]
 [ 2.44666147e+00]
 [ 2.44733238e+00]
 [ 2.44799566e+00]
 [ 2.44865227e+00]
 [ 2.44930029e+00]
 [ 2.44994164e+00]
 [ 2.45057487e+00]
 [ 2.45123720e+00]
 [ 2.45185518e+00]
 [ 2.45246601e+00]
 [ 2.45306921e+00]
 [ 2.45366430e+00]
 [ 2.45425224e+00]
 [ 2.45483208e+00]
 [ 2.45540500e+00]
 [ 2.45596933e+00]
 [ 2.45652676e+00]
 [ 2.45707655e+00]
 [ 2.45761776e+00]
 [ 2.45815182e+00]
 [ 2.45867777e+00]
 [ 2.45919609e+00]
 [ 2.45970726e+00]
 [ 2.46021032e+00]
 [ 2.46070552e+00]
 [ 2.46119308e+00]
 [ 2.46167278e+00]
 [ 2.46214485e+00]
 [ 2.46260905e+00]
 [ 2.46306539e+00]
 [ 2.46351385e+00]
 [ 2.46395469e+00]
 

# advanced auto grad Examples

In [50]:
import tensorflow as tf

# Define your model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(4,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1)
])

# Define some tensors
tensor1 = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
tensor2 = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

# Compute model output
with tf.GradientTape(persistent=True) as tape:
    tape.watch(tensor1)
    tape.watch(tensor2)

    # Concatenate the tensors along axis 0
    concatenated_tensor = tf.concat([tensor1, tensor2], axis=0)
    concatenated_tensor = tf.reshape(concatenated_tensor, (1,4,2))
    print(concatenated_tensor.shape)
    
    # Pass the concatenated tensor as input to the model
    model_output = model(concatenated_tensor)

# Compute gradients of the model output with respect to the individual tensors
gradient_tensor1 = tape.gradient(model_output, tensor1)
gradient_tensor2 = tape.gradient(model_output, tensor2)

print("Gradient of the model output with respect to tensor1:")
print(gradient_tensor1)

print("\nGradient of the model output with respect to tensor2:")
print(gradient_tensor2)


(1, 4, 2)
Gradient of the model output with respect to tensor1:
tf.Tensor(
[[-0.17236525  0.09680016]
 [-0.10308962 -0.23470856]], shape=(2, 2), dtype=float32)

Gradient of the model output with respect to tensor2:
tf.Tensor(
[[ 0.17332584  0.00868668]
 [-0.24331066  0.11009061]], shape=(2, 2), dtype=float32)
